### Where does the dataset come from? Download from yelp.com

### Businesses

In [14]:
import pandas as pd
import json
import re
import numpy as np

In [15]:
data = []

with open('yelp_dataset/yelp_academic_dataset_business.json') as f:

    for line in f:
        
        json_dict = json.loads(line)

        if json_dict['attributes']:
            if 'BusinessParking' in json_dict['attributes'].keys():

                json_dict['attributes']['BusinessParking'] = eval(json_dict['attributes']['BusinessParking'])

        data.append(json_dict)

In [16]:
businesses = pd.json_normalize(data)

In [17]:
businesses.columns = businesses.columns.str.split(('.')).str[-1]

In [18]:
businesses.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories', 'hours', 'ByAppointmentOnly',
       'BusinessAcceptsCreditCards', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'BikeParking',
       'RestaurantsPriceRange2', 'CoatCheck', 'RestaurantsTakeOut',
       'RestaurantsDelivery', 'Caters', 'WiFi', 'garage', 'street',
       'validated', 'lot', 'valet', 'WheelchairAccessible', 'HappyHour',
       'OutdoorSeating', 'HasTV', 'RestaurantsReservations', 'DogsAllowed',
       'Sunday', 'Alcohol', 'GoodForKids', 'BusinessParking',
       'RestaurantsAttire', 'Ambience', 'RestaurantsTableService',
       'RestaurantsGoodForGroups', 'DriveThru', 'attributes', 'NoiseLevel',
       'GoodForMeal', 'BusinessAcceptsBitcoin', 'Smoking', 'Music',
       'GoodForDancing', 'AcceptsInsurance', 'BestNights', 'BYOB', 'Corkage',
       'BYOBCorkage', 'HairSpecializesIn', 'O

In [19]:
columns = []

for column in businesses.columns:
    column = re.sub(r'(\w)([A-Z])', r'\1_\2', column)
    column = re.sub(r'Restaurants_', r'', column)
    column = re.sub(r'Business_', r'', column)
    column = re.sub(r'(Range)([0-9])', r'\1', column)
    columns.append(column.lower())

businesses.columns = columns

In [26]:
businesses.drop(
  [
    'attributes',
    'hair_specializes_in',
    'counter_service',
    'open24_hours',
    'dietary_restrictions',
    'accepts_insurance',
    'ages_allowed',
    'b_yo_bcorkage',
    'corkage',
    'smoking',
    'b_yo_b',
    'good_for_dancing',
    'coat_check',
    'by_appointment_only',
    'best_nights',
    'music',
    'drive_thru',
    'accepts_bitcoin',
    'dogs_allowed',
    'happy_hour',
    'wheelchair_accessible',
    'good_for_meal',
    'ambience',
    'business_parking',
    'address',
    'postal_code',
    'hours',
    'is_open',
    'monday',
    'tuesday',
    'wednesday',
    'thursday',
    'friday',
    'saturday',
    'sunday'
  ],
  axis=1, errors='ignore', inplace=True
)

In [21]:
businesses['alcohol'] = businesses['alcohol'].str.replace("u'", "").str.replace("'", "")
businesses['noise_level'] = businesses['noise_level'].str.replace("u'", "").str.replace("'", "")
businesses['attire'] = businesses['attire'].str.replace("u'", "").str.replace("'", "")
businesses['wi_fi'] = businesses['wi_fi'].str.replace("u'", "").str.replace("'", "")

In [22]:
def check_parking(row):
    if row['garage'] == True or row['street'] == True or row['validated'] == True or row['lot'] == True or row['valet'] == True:
        return 'True'
    if row['garage'] == False or row['street'] == False or row['validated'] == False or row['lot'] == False or row['valet'] == False:
        return 'False'
    else:
        return np.nan

In [13]:
businesses['parking_available'] = businesses.apply(check_parking, axis=1)

In [23]:
businesses.drop(['garage', 'street','validated','lot','valet'], axis=1, inplace=True)

In [27]:
businesses.columns

Index(['business_id', 'name', 'city', 'state', 'latitude', 'longitude',
       'stars', 'review_count', 'categories', 'accepts_credit_cards',
       'bike_parking', 'price_range', 'take_out', 'delivery', 'caters',
       'wi_fi', 'outdoor_seating', 'has_tv', 'reservations', 'alcohol',
       'good_for_kids', 'parking', 'attire', 'table_service',
       'good_for_groups', 'noise_level'],
      dtype='object')

### We have created a table with all businesses and the attributes we want to have a look at

### Businesses in Pennsylvania

In [33]:
pennsylvania = businesses[businesses['state'] == 'PA'].copy()

### Restaurants in Pennsylvania

In [35]:
pennsylvania.dropna(subset='categories', inplace=True)
pennsylvania.reset_index(drop=True, inplace=True)

In [47]:
# Is there a faster ways to do this?

categories =[
    'Coffee & Tea',
    'Bistros',
    'Breakfast & Brunch',
    'Cafes',
    'French',
    'Greek',
    'Italian',
    'Mexican',
    'Tacos',
    'Egyptian',
    'Pizza',
    'Soup',
    'Sushi Bars',
    'Vegetarian',
    'Waffles',
    'Food',
    'Restaurants',
    'Bars'
]

restaurants = pd.DataFrame()

for index, item in pennsylvania['categories'].items():

    for category in categories:
        if category in item:
            restaurants = pd.concat([restaurants, pennsylvania[index:index+1]])
            break

In [49]:
with open('tables/restaurants_pennsylvania.csv', 'w') as f:
    restaurants.to_csv(f, mode='a', header=True, index=False)

### We have created a table with all restaurants in Pennsylvania and saved it in a .csv for later analysis

### Now we load the table with all reviews and create a .csv which only contains reviews from restaurants in Pennsylvania

In [50]:
businesses_ids = restaurants['business_id'].to_list()

column_names = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']

chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_review.json', lines=True, chunksize=100000)

with open('tables/reviews_pennsylvania.csv', 'w') as f:
    header = ','.join(column_names)
    f.write(header + '\n')

    for chunk in chunks:
        reviews = pd.DataFrame(chunk)
        
        reviews[reviews['business_id'].isin(businesses_ids)].to_csv(f, mode='a', header=False, index=False)

In [ ]:
# Replace values in columns by dummies

columns = []
stop_list = [
    'business_id',
    'name',
    'city',
    'state',
    'latitude',
    'longitude',
    'stars',
    'review_count',
    'categories',
    'monday',
    'tuesday',
    'wednesday',
    'thursday',
    'friday',
    'saturday',
    'sunday'
]

for column in restaurants.columns:
    if column not in stop_list:  
        columns.append(column)

for column in columns:
    restaurants[column].replace('True', 1, inplace=True)
    restaurants[column].replace('False', 0, inplace=True)
    restaurants[column].replace('nan', np.NaN, inplace=True)
    restaurants[column].replace('None', np.NaN, inplace=True)
    restaurants[column].replace('none', np.NaN, inplace=True)
    restaurants[column].replace('casual', 0, inplace=True)
    restaurants[column].replace('formal', 1, inplace=True)
    restaurants[column].replace('dressy', 2, inplace=True)
    restaurants[column].replace('full_bar', 0, inplace=True)
    restaurants[column].replace('beer_and_wine', 1, inplace=True)
    restaurants[column].replace('average', 0, inplace=True)
    restaurants[column].replace('quiet', 1, inplace=True)
    restaurants[column].replace('loud', 2, inplace=True)
    restaurants[column].replace('very_loud', 3, inplace=True)
    restaurants[column].replace('no', 0, inplace=True)
    restaurants[column].replace('free', 1, inplace=True)
    restaurants[column].replace('paid', 2, inplace=True)